In [ ]:
#R2
import networkx as nx
import matplotlib.pyplot as plt
import time
from queue import Queue

# Global Variables
NUM_STATIONS = 10
NUM_ROBOTS = 2
MAX_OUT_TRAY_CAPACITY = 5
SIMULATION_TIME = 1440  # 24 hours = 1440 ticks

stations = [7, 6, 2, 4, 3, 2]  # Available stations
order_id = 1  # Order ID counter
order_queue = []

def create_warehouse_graph():
    G = nx.Graph()
    G.add_edges_from([(3, 9, {'weight': 2}),(1, 4, {'weight': 1}), (1, 9,{'weight': 1}),
                      (4,9,{'weight': 1}), (9,2, {'weight': 1}), (9,7, {'weight': 1}),
                      (2,7, {'weight': 1}), (2,5, {'weight': 1}), (7,5, {'weight': 1}),
                      (7,8,{'weight': 2}), (5,6, {'weight': 1}), (5,8, {'weight': 1}),
                      (5,10, {'weight': 2})])
    return G

class Robot:
    def __init__(self, id, start_position):
        self.id = id
        self.current_position = start_position
        self.in_tray = Queue()
        self.out_tray = []
        self.route = [1]
        self.available = True #Flag to determine if the tobot is available
        self.wait_counter = 0 #Stores the weight of the edge > 1
        self.next_position = None # Stores the next position for the robot waiting for movement
        self.returning_to_dispatch = False  # New flag to track if robot is returning to dispatch

    def process_orders(self, warehouse_graph, tick):
        if self.wait_counter > 0: #Condition for robots waiting to move due to high weighted edge
            self.wait_counter -= 1 #Decreases the counter after each tick
            print(f"Clock Tick {tick}: Robot {self.id} waiting at {self.current_position} ({self.wait_counter + 1} ticks remaining)")
            if self.wait_counter == 0 and self.next_position: #Moves after the wait time
                print(f"Clock Tick {tick}: Robot {self.id} moved from {self.current_position} to {self.next_position}")
                self.current_position = self.next_position #Updates the robot's position after it has moved
                self.route.append(self.next_position) #Appends the new position of the robot to keep track of its movements
                self.next_position = None
            return


        if self.returning_to_dispatch and self.out_tray:  # Check if robot should return to dispatch
            path = nx.astar_path(warehouse_graph, source=self.current_position, target=1, weight='weight') #Finds the shortest path from the source to the target
            if len(path) > 1: #Determines if the robot is at the target already
                next_position = path[1] #If not it records the next node to move to
                edge_weight = warehouse_graph[self.current_position][next_position]['weight'] #Retrieves the edge weight of the path

                if edge_weight == 2:
                    self.wait_counter = 1 #The robot has to wait for 1 tick to move
                    self.next_position = next_position
                    print(f"Clock Tick {tick}: Robot {self.id} starting final return movement from {self.current_position} to {next_position} (weight: {edge_weight})")
                else:
                    print(f"Clock Tick {tick}: Robot {self.id} moved from {self.current_position} to {next_position} during final return")
                    self.current_position = next_position
                    self.route.append(next_position) #Appends the node to the path of the robot
            elif self.current_position == 1: #If the robot reached the dispatch center
                print(f"Clock Tick {tick}: Robot {self.id} reached dispatch center and emptied final out-tray: {self.out_tray}")
                self.out_tray.clear() #Clears out the out-tray of the robot
                self.available = False
                self.returning_to_dispatch = False
            return

        elif (not self.in_tray.empty()) and self.available == True: #If the robot has to pickup orders
            order_id, station_id = self.in_tray.queue[0] #Retrieves the order details from the order queue

            path = nx.astar_path(warehouse_graph, source=self.current_position, target=station_id, weight='weight') #Finds the shortest path from the source to the target
            if station_id == 1 and self.current_position == 1: #If the robot is already at the dispatch center
                self.in_tray.get() #Retrieves the firs item from the queue
                self.out_tray.append((order_id, station_id)) #Adds to the out-Tray of the Robot
                print(f"Clock Tick {tick}: Robot {self.id} immediately processed Order {order_id} at Station {station_id} (dispatch center)")
                print(f"Robot {self.id} - In-tray: {list(self.in_tray.queue)}, Out-tray: {self.out_tray}")
                return

            if len(path) > 1: #If the robot has to travel to pivkup orders
                next_position = path[1] #Stores the next node to travel to
                edge_weight = warehouse_graph[self.current_position][next_position]['weight'] #Retrieves the weight of the edge

                if edge_weight == 2: #Sets the wait counter for the robot
                    self.wait_counter = 1
                    self.next_position = next_position
                    print(f"Clock Tick {tick}: Robot {self.id} starting movement from {self.current_position} to {next_position} (weight: {edge_weight})")
                else:
                    print(f"Clock Tick {tick}: Robot {self.id} moved from {self.current_position} to {next_position} (weight: {edge_weight})")
                    self.current_position = next_position  #Updates the robot's position after it has moved
                    self.route.append(next_position) #Appends the new position of the robot to keep track of its movements

            elif self.current_position == station_id:
                self.in_tray.get()
                self.out_tray.append((order_id, station_id))
                print(f"Clock Tick {tick}: Robot {self.id} picked up Order {order_id} at Station {station_id} Out-tray = {self.out_tray}")

                # Check if this was the last order
                if self.in_tray.empty():
                    self.returning_to_dispatch = True  # Set flag to return to dispatch
                return
        else:
            # If no more orders in in-tray and has items in out-tray, return to dispatch
            if self.in_tray.empty() and self.out_tray and not self.returning_to_dispatch:
                self.returning_to_dispatch = True
                print(f"Clock Tick {tick}: Robot {self.id} finished all orders, returning to dispatch with out-tray: {self.out_tray}")
            return

    def assign_order(self, order_id, station_id):
        self.in_tray.put((order_id, station_id))

def generate_orders(tick): #Generating orders based on Tutorial !
    global order_id, stations, order_queue # Declaring global variable
    if not stations: #Checks for stations
        print("No more stations available for order generation")
    else:
        station_id = stations.pop(0) # Returns the first station from the list
        print(f"Clock Tick {tick}: New Order {order_id} generated at Station {station_id}")
        order_queue.append((order_id, station_id)) # Adds the order ID and the station ID to the order queue
        order_id += 1 #Increments order ID for the next order

def assign_orders_to_robots(robots, warehouse_graph, tick):
    global order_queue #Declaring global variable
    if not order_queue:
        return  # No orders to assign

    available_robots = [robot for robot in robots if robot.available == True and robot.in_tray.qsize() < MAX_OUT_TRAY_CAPACITY] #Lists all the robots with set conditions to be available

    if available_robots:
        order_id, station_id = order_queue.pop(0)  # Get first order in the order queue
        nearest_robot = min(available_robots, key=lambda r: nx.astar_path_length(warehouse_graph, r.current_position, station_id, weight='weight')) #Finds the nearest available robots from the station considering the weight of the edge as well
        nearest_robot.assign_order(order_id, station_id)
        print(f"Clock Tick {tick}: Assigned Order {order_id} to Robot {nearest_robot.id} In-Tray = {list(nearest_robot.in_tray.queue)} Out-Tray = {nearest_robot.out_tray}")
    else:
        print(f"Clock Tick {tick}: No available robots to handle orders. Current order queue size: {len(order_queue)}")

def plot_warehouse_graph(G, robots, tick):
    plt.figure(figsize=(8, 6))

    # Use spring layout for visualization
    pos = nx.spring_layout(G, seed=42)

    # Draw nodes
    nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=500)

    # Draw edges with weights
    nx.draw_networkx_edges(G, pos, width=2)
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels)

    # Draw robot positions
    robot_positions = {robot.current_position for robot in robots}
    nx.draw_networkx_nodes(G, pos, nodelist=robot_positions, node_color='red', node_size=600, label="Robots")

    # Draw labels
    nx.draw_networkx_labels(G, pos, font_size=12)

    plt.title(f"Warehouse Layout (Clock Tick: {tick})")
    plt.legend(["Stations", "Robots"], loc="upper left")
    plt.show()
    time.sleep(0.5)  # Pause for animation effect

def simulate():
    global G, order_queue
    G = create_warehouse_graph()
    robots = [Robot(i, 1) for i in range(1, NUM_ROBOTS + 1)]
    for tick in range(1, SIMULATION_TIME + 1):
        generate_orders(tick)  # Generate one order per tick
        assign_orders_to_robots(robots, G, tick)  # Assign orders per tick

        for robot in robots:
            robot.process_orders(G, tick)  # Move robots one step
        print("\n")

        # Stop simulation early if all robots are idle and there are no orders in queue
        if all(robot.in_tray.empty() and not robot.out_tray for robot in robots) and not order_queue:
            print(f"Clock Tick {tick}: No pending orders, ending simulation.")
            break

    print("Simulation complete.")
    for robot in robots:
        print(f"Robot {robot.id} Final Path: {robot.route} Final Out-Tray = {robot.out_tray}")
    # plot_warehouse_graph(G, robots, tick)

# Run the simulation
simulate()

In [ ]:
#R3
import networkx as nx
import random
from queue import Queue

# Define constants
NUM_STATIONS = 100
NUM_ROBOTS = 20
MAX_OUT_TRAY_CAPACITY = 5
SIMULATION_TIME = 1440  # 24 hours = 1440 ticks


stations = [i for i in range(1, NUM_STATIONS + 1)]  # Available stations
order_id = 1  # Order ID counter
order_queue = []  # Initialize the order queue
robot_timings = {} # Stores the clocked times for each robot with its times


def create_warehouse_graph():
    G = nx.gnp_random_graph(101, 0.06)
    mapping = {old_label: old_label + 1 for old_label in G.nodes()}
    G = nx.relabel_nodes(G, mapping)

    prev_node = None
    for node in nx.nodes(G):
        if prev_node is not None:
            G.add_edge(prev_node, node)
        prev_node = node

    # for u, v in list(G.edges()):
    #     if random.randint(0, 100) > 50:
    #         G.remove_edge(u, v)

    if not nx.is_connected(G):
        G = nx.connected_watts_strogatz_graph(G.number_of_nodes(), 2, 0.8, tries=100, create_using=G)

    glist = list(G.edges())
    random.shuffle(glist)
    count = 0
    for u, v in glist:
        weight = 2 if count < len(glist) * 0.3 else 1
        G.add_edge(u, v, weight=weight)
        count += 1

    return G


class Robot:
    def __init__(self, id, start_position):
        self.id = id
        self.current_position = start_position
        self.in_tray = Queue()
        self.out_tray = []
        self.route = [1]
        self.wait_in_tray = 0
        self.wait_counter = 0
        self.next_position = None
        self.returning_to_dispatch = False  # New flag to track if robot is returning to dispatch
        self.proccessing_time = {} #Records all the completion times for each order

    def process_orders(self, warehouse_graph, tick):
        if self.wait_counter > 0: #Condition for robots waiting to move due to high weighted edge
            self.wait_counter -= 1 #Decreases the counter after each tick
            print(f"Clock Tick {tick}: Robot {self.id} waiting at {self.current_position} ({self.wait_counter + 1} ticks remaining)")
            if self.wait_counter == 0 and self.next_position: #Moves after the wait time
                print(f"Clock Tick {tick}: Robot {self.id} moved from {self.current_position} to {self.next_position}")
                self.current_position = self.next_position #Updates the robot's position after it has moved
                self.route.append(self.next_position) #Appends the new position of the robot to keep track of its movements
                self.next_position = None
            return

        if self.returning_to_dispatch and len(self.out_tray) >= MAX_OUT_TRAY_CAPACITY: # Check if robot should return to dispatch
            path = nx.astar_path(warehouse_graph, source=self.current_position, target=1, weight='weight') #Finds the shortest path from the source to the target
            if len(path) > 1: #Determines if the robot is at the target already
                next_position = path[1] #If not it records the next node to move to
                edge_weight = warehouse_graph[self.current_position][next_position]['weight'] #Retrieves the edge weight of the path

                if edge_weight == 2:
                    self.wait_counter = 1 #The robot has to wait for 1 tick to move
                    self.next_position = next_position
                    print(f"Clock Tick {tick}: Robot {self.id} starting final return movement from {self.current_position} to {next_position} (weight: {edge_weight})")
                else:
                    print(f"Clock Tick {tick}: Robot {self.id} moved from {self.current_position} to {next_position} during final return")
                    self.current_position = next_position
                    self.route.append(next_position) #Appends the node to the path of the robot
            elif self.current_position == 1: #If the robot reached the dispatch center
                print(f"Clock Tick {tick}: Robot {self.id} reached dispatch center and emptied final out-tray: {self.out_tray}")
                for order_id, station_id in self.out_tray: #Function to record the time of completion
                    if order_id in self.proccessing_time:
                        start = self.proccessing_time[order_id]
                        time = tick - start
                    if self.id not in robot_timings: #Checks if the id already exists in the dictionary
                        robot_timings[self.id] = []
                    robot_timings[self.id].append(time) #Appends the time for the respective robot
                    del self.proccessing_time[order_id]
                self.out_tray.clear() #Clears out the out-tray of the robot
                self.returning_to_dispatch = False
            return

        elif (not self.in_tray.empty()): #If there are some orders in the In-tray
            order_id, station_id = self.in_tray.queue[0] #Retrieves the order details from the order queue

            path = nx.astar_path(warehouse_graph, source=self.current_position, target=station_id, weight='weight') #Finds the shortest path from the source to the target
            if station_id == 1 and self.current_position == 1: #If the robot is already at the dispatch center
                self.in_tray.get() #Retrieves the firs item from the queue
                self.out_tray.append((order_id, station_id)) #Adds to the out-Tray of the Robot
                print(f"Clock Tick {tick}: Robot {self.id} immediately processed Order {order_id} at Station {station_id} (dispatch center)")
                print(f"Robot {self.id} - In-tray: {list(self.in_tray.queue)}, Out-tray: {self.out_tray}")
                return

            if len(path) > 1: #If the robot has to travel to pivkup orders
                next_position = path[1] #Stores the next node to travel to
                edge_weight = warehouse_graph[self.current_position][next_position]['weight'] #Retrieves the weight of the edge

                if edge_weight == 2: #Sets the wait counter for the robot
                    self.wait_counter = 1
                    self.next_position = next_position
                    print(f"Clock Tick {tick}: Robot {self.id} starting movement from {self.current_position} to {next_position} (weight: {edge_weight})")
                else:
                    print(f"Clock Tick {tick}: Robot {self.id} moved from {self.current_position} to {next_position} (weight: {edge_weight})")
                    self.current_position = next_position #Updates the robot's position after it has moved
                    self.route.append(next_position) #Appends the new position of the robot to keep track of its movements

            elif self.current_position == station_id:
                self.in_tray.get()
                self.out_tray.append((order_id, station_id))
                print(f"Clock Tick {tick}: Robot {self.id} picked up Order {order_id} at Station {station_id} Out-tray = {self.out_tray}")

                # Check if this was the last order
                if self.in_tray.empty():
                    self.returning_to_dispatch = True  # Set flag to return to dispatch
                return
        else:
            # If no more orders in in-tray and has items in out-tray for some time
            if self.in_tray.empty() and self.out_tray and not self.returning_to_dispatch:
                if self.wait_in_tray < 4:  # Wait for 4 ticks
                    self.wait_in_tray += 1
                    print(f"Clock Tick {tick}: Robot {self.id} waiting before return to dispatch ({self.empty_tray_wait}/4)")
                return
            else:
                if self.current_position != 1:
                    self.returning_to_dispatch = True
                    self.empty_tray_wait = 0  # Reset the counter
                    print(f"Clock Tick {tick}: Robot {self.id} finished waiting, returning to dispatch with out-tray: {self.out_tray}")
        return

    def assign_order(self, order_id, station_id, tick):
        self.in_tray.put((order_id, station_id))
        self.proccessing_time[order_id] = tick


def generate_orders(tick):
    global order_id, stations, order_queue # Declaring global variable
    num_orders = random.randint(8, 10) #Variable count of orders generated per clock tick
    available_stations = [s for s in stations if s > 1]  # Exclude stations 0 and 1
    for i in range(num_orders):
        station_id = random.choice(available_stations) #Selects a random station from the list of available stations
        print(f"Clock Tick {tick}: New Order {order_id} generated at Station {station_id}")
        order_queue.append((order_id, station_id)) # Adds the order ID and the station ID to the order queue
        order_id += 1 #Increments order ID for the next order


def assign_orders_to_robots(robots, warehouse_graph, tick):
    global order_queue #Declaring global variable
    if not order_queue:
        return

    available_robots = [robot for robot in robots if robot.in_tray.qsize() < MAX_OUT_TRAY_CAPACITY] #Lists all the robots with set conditions to be available

    if available_robots:
        order_id, station_id = order_queue.pop(0) # Get first order in the order queue
        nearest_robot = min(
            available_robots,
            key=lambda r: nx.astar_path_length(warehouse_graph, r.current_position, station_id, weight='weight') #Finds the nearest available robots from the station considering the weight of the edge as well
        )
        nearest_robot.assign_order(order_id, station_id, tick)
        print(f"Clock Tick {tick}: Assigned Order {order_id} to Robot {nearest_robot.id} In-Tray = {list(nearest_robot.in_tray.queue)} Out-Tray = {nearest_robot.out_tray}")
    else:
        print(f"Clock Tick {tick}: No available robots to handle orders. Current order queue size: {len(order_queue)}")


def simulate():
    global tick
    total_orders = 0
    total_time = 0
    G = create_warehouse_graph()
    robots = [Robot(i, 1) for i in range(1, NUM_ROBOTS + 1)]

    for tick in range(1, SIMULATION_TIME + 1):
        generate_orders(tick)  # Generate one order per tick
        assign_orders_to_robots(robots, G, tick) # Assign orders per tick

        for robot in robots:
            robot.process_orders(G, tick) # Move robots one step

        print("\n")
        # Stop simulation early if all robots are idle and there are no orders in queue
        if all(robot.in_tray.empty() and not robot.out_tray for robot in robots) and not order_queue:
            print(f"Clock Tick {tick}: No pending orders, ending simulation.")
            break

    print("\nSimulation complete.")
    for id, time in robot_timings.items():
        if time:
            avg = sum(time)/ len(time)
            total_orders += len(time)
            total_time += avg
            print(f"Robot {id}")
            print(f"  Average Processing Time: {avg:.2f} ticks")

    print(f"Total orders completed in 24-hour cycle: {total_orders} and the average processing time per robot would be {total_time/NUM_ROBOTS:.2f} ticks")

# Run the simulation
simulate()

In [ ]:
#R4
import networkx as nx
import random
from queue import Queue

# Define constants
NUM_STATIONS = 100
NUM_ROBOTS = 40
MAX_OUT_TRAY_CAPACITY = 5
SIMULATION_TIME = 1440  # 24 hours = 1440 ticks


stations = [i for i in range(1, NUM_STATIONS + 1)]  # Available stations
order_id = 1  # Order ID counter
order_queue = []  # Initialize the order queue
robot_timings = {} # Stores the clocked times for each robot with its times


def create_warehouse_graph():
    G = nx.gnp_random_graph(101, 0.06)
    mapping = {old_label: old_label + 1 for old_label in G.nodes()}
    G = nx.relabel_nodes(G, mapping)

    prev_node = None
    for node in nx.nodes(G):
        if prev_node is not None:
            G.add_edge(prev_node, node)
        prev_node = node

    # for u, v in list(G.edges()):
    #     if random.randint(0, 100) > 50:
    #         G.remove_edge(u, v)

    if not nx.is_connected(G):
        G = nx.connected_watts_strogatz_graph(G.number_of_nodes(), 2, 0.8, tries=100, create_using=G)

    glist = list(G.edges())
    random.shuffle(glist)
    count = 0
    for u, v in glist:
        weight = 2 if count < len(glist) * 0.3 else 1
        G.add_edge(u, v, weight=weight)
        count += 1

    return G


class Robot:
    def __init__(self, id, start_position):
        self.id = id
        self.current_position = start_position
        self.in_tray = Queue()
        self.out_tray = []
        self.route = [1]
        self.wait_in_tray = 0
        self.wait_counter = 0
        self.next_position = None
        self.returning_to_dispatch = False  # New flag to track if robot is returning to dispatch
        self.proccessing_time = {} #Records all the completion times for each order

    def process_orders(self, warehouse_graph, tick):
        if self.wait_counter > 0: #Condition for robots waiting to move due to high weighted edge
            self.wait_counter -= 1 #Decreases the counter after each tick
            print(f"Clock Tick {tick}: Robot {self.id} waiting at {self.current_position} ({self.wait_counter + 1} ticks remaining)")
            if self.wait_counter == 0 and self.next_position: #Moves after the wait time
                print(f"Clock Tick {tick}: Robot {self.id} moved from {self.current_position} to {self.next_position}")
                self.current_position = self.next_position #Updates the robot's position after it has moved
                self.route.append(self.next_position) #Appends the new position of the robot to keep track of its movements
                self.next_position = None
            return

        if self.returning_to_dispatch and len(self.out_tray) >= MAX_OUT_TRAY_CAPACITY: # Check if robot should return to dispatch
            path = nx.astar_path(warehouse_graph, source=self.current_position, target=1, weight='weight') #Finds the shortest path from the source to the target
            if len(path) > 1: #Determines if the robot is at the target already
                next_position = path[1] #If not it records the next node to move to
                edge_weight = warehouse_graph[self.current_position][next_position]['weight'] #Retrieves the edge weight of the path

                if edge_weight == 2:
                    self.wait_counter = 1 #The robot has to wait for 1 tick to move
                    self.next_position = next_position
                    print(f"Clock Tick {tick}: Robot {self.id} starting final return movement from {self.current_position} to {next_position} (weight: {edge_weight})")
                else:
                    print(f"Clock Tick {tick}: Robot {self.id} moved from {self.current_position} to {next_position} during final return")
                    self.current_position = next_position
                    self.route.append(next_position) #Appends the node to the path of the robot
            elif self.current_position == 1: #If the robot reached the dispatch center
                print(f"Clock Tick {tick}: Robot {self.id} reached dispatch center and emptied final out-tray: {self.out_tray}")
                for order_id, station_id in self.out_tray: #Function to record the time of completion
                    if order_id in self.proccessing_time:
                        start = self.proccessing_time[order_id]
                        time = tick - start
                    if self.id not in robot_timings: #Checks if the id already exists in the dictionary
                        robot_timings[self.id] = []
                    robot_timings[self.id].append(time) #Appends the time for the respective robot
                    del self.proccessing_time[order_id]
                self.out_tray.clear() #Clears out the out-tray of the robot
                self.returning_to_dispatch = False
            return

        elif (not self.in_tray.empty()): #If there are some orders in the In-tray
            order_id, station_id = self.in_tray.queue[0] #Retrieves the order details from the order queue

            path = nx.astar_path(warehouse_graph, source=self.current_position, target=station_id, weight='weight') #Finds the shortest path from the source to the target
            if station_id == 1 and self.current_position == 1: #If the robot is already at the dispatch center
                self.in_tray.get() #Retrieves the firs item from the queue
                self.out_tray.append((order_id, station_id)) #Adds to the out-Tray of the Robot
                print(f"Clock Tick {tick}: Robot {self.id} immediately processed Order {order_id} at Station {station_id} (dispatch center)")
                print(f"Robot {self.id} - In-tray: {list(self.in_tray.queue)}, Out-tray: {self.out_tray}")
                return

            if len(path) > 1: #If the robot has to travel to pivkup orders
                next_position = path[1] #Stores the next node to travel to
                edge_weight = warehouse_graph[self.current_position][next_position]['weight'] #Retrieves the weight of the edge

                if edge_weight == 2: #Sets the wait counter for the robot
                    self.wait_counter = 1
                    self.next_position = next_position
                    print(f"Clock Tick {tick}: Robot {self.id} starting movement from {self.current_position} to {next_position} (weight: {edge_weight})")
                else:
                    print(f"Clock Tick {tick}: Robot {self.id} moved from {self.current_position} to {next_position} (weight: {edge_weight})")
                    self.current_position = next_position #Updates the robot's position after it has moved
                    self.route.append(next_position) #Appends the new position of the robot to keep track of its movements

            elif self.current_position == station_id:
                self.in_tray.get()
                self.out_tray.append((order_id, station_id))
                print(f"Clock Tick {tick}: Robot {self.id} picked up Order {order_id} at Station {station_id} Out-tray = {self.out_tray}")

                # Check if this was the last order
                if self.in_tray.empty():
                    self.returning_to_dispatch = True  # Set flag to return to dispatch
                return
        else:
            # If no more orders in in-tray and has items in out-tray for some time
            if self.in_tray.empty() and self.out_tray and not self.returning_to_dispatch:
                if self.wait_in_tray < 4:  # Wait for 4 ticks
                    self.wait_in_tray += 1
                    print(f"Clock Tick {tick}: Robot {self.id} waiting before return to dispatch ({self.empty_tray_wait}/4)")
                return
            else:
                if self.current_position != 1:
                    self.returning_to_dispatch = True
                    self.empty_tray_wait = 0  # Reset the counter
                    print(f"Clock Tick {tick}: Robot {self.id} finished waiting, returning to dispatch with out-tray: {self.out_tray}")
        return

    def assign_order(self, order_id, station_id, tick):
        self.in_tray.put((order_id, station_id))
        self.proccessing_time[order_id] = tick


def generate_orders(tick):
    global order_id, stations, order_queue # Declaring global variable
    num_orders = random.randint(8, 10) #Variable count of orders generated per clock tick
    available_stations = [s for s in stations if s > 1]  # Exclude stations 0 and 1
    for i in range(num_orders):
        station_id = random.choice(available_stations) #Selects a random station from the list of available stations
        print(f"Clock Tick {tick}: New Order {order_id} generated at Station {station_id}")
        order_queue.append((order_id, station_id)) # Adds the order ID and the station ID to the order queue
        order_id += 1 #Increments order ID for the next order


def assign_orders_to_robots(robots, warehouse_graph, tick):
    global order_queue #Declaring global variable
    if not order_queue:
        return

    available_robots = [robot for robot in robots if robot.in_tray.qsize() < MAX_OUT_TRAY_CAPACITY] #Lists all the robots with set conditions to be available

    if available_robots:
        order_id, station_id = order_queue.pop(0) # Get first order in the order queue
        nearest_robot = min(
            available_robots,
            key=lambda r: nx.astar_path_length(warehouse_graph, r.current_position, station_id, weight='weight') #Finds the nearest available robots from the station considering the weight of the edge as well
        )
        nearest_robot.assign_order(order_id, station_id, tick)
        print(f"Clock Tick {tick}: Assigned Order {order_id} to Robot {nearest_robot.id} In-Tray = {list(nearest_robot.in_tray.queue)} Out-Tray = {nearest_robot.out_tray}")
    else:
        print(f"Clock Tick {tick}: No available robots to handle orders. Current order queue size: {len(order_queue)}")


def simulate():
    global tick
    total_orders = 0
    total_time = 0
    G = create_warehouse_graph()
    robots = [Robot(i, 1) for i in range(1, NUM_ROBOTS + 1)]

    for tick in range(1, SIMULATION_TIME + 1):
        generate_orders(tick)  # Generate one order per tick
        assign_orders_to_robots(robots, G, tick) # Assign orders per tick

        for robot in robots:
            robot.process_orders(G, tick) # Move robots one step

        print("\n")
        # Stop simulation early if all robots are idle and there are no orders in queue
        if all(robot.in_tray.empty() and not robot.out_tray for robot in robots) and not order_queue:
            print(f"Clock Tick {tick}: No pending orders, ending simulation.")
            break

    print("\nSimulation complete.")
    for id, time in robot_timings.items():
        if time:
            avg = sum(time)/ len(time)
            total_orders += len(time)
            total_time += avg
            print(f"Robot {id}")
            print(f"  Average Processing Time: {avg:.2f} ticks")

    print(f"Total orders completed in 24-hour cycle: {total_orders} and the average processing time per robot would be {total_time/NUM_ROBOTS:.2f} ticks")

# Run the simulation
simulate()

In [ ]:
#R5
import networkx as nx
import random
from queue import Queue

# Define constants
NUM_STATIONS = 100
NUM_ROBOTS = 40
MAX_OUT_TRAY_CAPACITY = 5
SIMULATION_TIME = 1440  # 24 hours = 1440 ticks


stations = [i for i in range(1, NUM_STATIONS + 1)]  # Available stations
order_id = 1  # Order ID counter
order_queue = []  # Initialize the order queue
robot_timings = {} # Stores the clocked times for each robot with its times


def create_warehouse_graph():
    G = nx.gnp_random_graph(101, 0.06)
    mapping = {old_label: old_label + 1 for old_label in G.nodes()}
    G = nx.relabel_nodes(G, mapping)

    prev_node = None
    for node in nx.nodes(G):
        if prev_node is not None:
            G.add_edge(prev_node, node)
        prev_node = node

    for u, v in list(G.edges()):
        if random.randint(0, 100) > 50:
            G.remove_edge(u, v)

    if not nx.is_connected(G):
        G = nx.connected_watts_strogatz_graph(G.number_of_nodes(), 2, 0.8, tries=100, create_using=G)

    glist = list(G.edges())
    random.shuffle(glist)
    count = 0
    for u, v in glist:
        weight = 2 if count < len(glist) * 0.3 else 1
        G.add_edge(u, v, weight=weight)
        count += 1

    return G


class Robot:
    def __init__(self, id, start_position):
        self.id = id
        self.current_position = start_position
        self.in_tray = Queue()
        self.out_tray = []
        self.route = [1]
        self.wait_in_tray = 0
        self.wait_counter = 0
        self.next_position = None
        self.returning_to_dispatch = False  # New flag to track if robot is returning to dispatch
        self.proccessing_time = {} #Records all the completion times for each order

    def process_orders(self, warehouse_graph, tick):
        if self.wait_counter > 0: #Condition for robots waiting to move due to high weighted edge
            self.wait_counter -= 1 #Decreases the counter after each tick
            print(f"Clock Tick {tick}: Robot {self.id} waiting at {self.current_position} ({self.wait_counter + 1} ticks remaining)")
            if self.wait_counter == 0 and self.next_position: #Moves after the wait time
                print(f"Clock Tick {tick}: Robot {self.id} moved from {self.current_position} to {self.next_position}")
                self.current_position = self.next_position #Updates the robot's position after it has moved
                self.route.append(self.next_position) #Appends the new position of the robot to keep track of its movements
                self.next_position = None
            return

        if self.returning_to_dispatch and len(self.out_tray) >= MAX_OUT_TRAY_CAPACITY: # Check if robot should return to dispatch
            path = nx.astar_path(warehouse_graph, source=self.current_position, target=1, weight='weight') #Finds the shortest path from the source to the target
            if len(path) > 1: #Determines if the robot is at the target already
                next_position = path[1] #If not it records the next node to move to
                edge_weight = warehouse_graph[self.current_position][next_position]['weight'] #Retrieves the edge weight of the path

                if edge_weight == 2:
                    self.wait_counter = 1 #The robot has to wait for 1 tick to move
                    self.next_position = next_position
                    print(f"Clock Tick {tick}: Robot {self.id} starting final return movement from {self.current_position} to {next_position} (weight: {edge_weight})")
                else:
                    print(f"Clock Tick {tick}: Robot {self.id} moved from {self.current_position} to {next_position} during final return")
                    self.current_position = next_position
                    self.route.append(next_position) #Appends the node to the path of the robot
            elif self.current_position == 1: #If the robot reached the dispatch center
                print(f"Clock Tick {tick}: Robot {self.id} reached dispatch center and emptied final out-tray: {self.out_tray}")
                for order_id, station_id in self.out_tray: #Function to record the time of completion
                    if order_id in self.proccessing_time:
                        start = self.proccessing_time[order_id]
                        time = tick - start
                    if self.id not in robot_timings: #Checks if the id already exists in the dictionary
                        robot_timings[self.id] = []
                    robot_timings[self.id].append(time) #Appends the time for the respective robot
                    del self.proccessing_time[order_id]
                self.out_tray.clear() #Clears out the out-tray of the robot
                self.returning_to_dispatch = False
            return

        elif (not self.in_tray.empty()): #If there are some orders in the In-tray
            order_id, station_id = self.in_tray.queue[0] #Retrieves the order details from the order queue

            path = nx.astar_path(warehouse_graph, source=self.current_position, target=station_id, weight='weight') #Finds the shortest path from the source to the target
            if station_id == 1 and self.current_position == 1: #If the robot is already at the dispatch center
                self.in_tray.get() #Retrieves the firs item from the queue
                self.out_tray.append((order_id, station_id)) #Adds to the out-Tray of the Robot
                print(f"Clock Tick {tick}: Robot {self.id} immediately processed Order {order_id} at Station {station_id} (dispatch center)")
                print(f"Robot {self.id} - In-tray: {list(self.in_tray.queue)}, Out-tray: {self.out_tray}")
                return

            if len(path) > 1: #If the robot has to travel to pivkup orders
                next_position = path[1] #Stores the next node to travel to
                edge_weight = warehouse_graph[self.current_position][next_position]['weight'] #Retrieves the weight of the edge

                if edge_weight == 2: #Sets the wait counter for the robot
                    self.wait_counter = 1
                    self.next_position = next_position
                    print(f"Clock Tick {tick}: Robot {self.id} starting movement from {self.current_position} to {next_position} (weight: {edge_weight})")
                else:
                    print(f"Clock Tick {tick}: Robot {self.id} moved from {self.current_position} to {next_position} (weight: {edge_weight})")
                    self.current_position = next_position #Updates the robot's position after it has moved
                    self.route.append(next_position) #Appends the new position of the robot to keep track of its movements

            elif self.current_position == station_id:
                self.in_tray.get()
                self.out_tray.append((order_id, station_id))
                print(f"Clock Tick {tick}: Robot {self.id} picked up Order {order_id} at Station {station_id} Out-tray = {self.out_tray}")

                # Check if this was the last order
                if self.in_tray.empty():
                    self.returning_to_dispatch = True  # Set flag to return to dispatch
                return
        else:
            # If no more orders in in-tray and has items in out-tray for some time
            if self.in_tray.empty() and self.out_tray and not self.returning_to_dispatch:
                if self.wait_in_tray < 4:  # Wait for 4 ticks
                    self.wait_in_tray += 1
                    print(f"Clock Tick {tick}: Robot {self.id} waiting before return to dispatch ({self.empty_tray_wait}/4)")
                return
            else:
                if self.current_position != 1:
                    self.returning_to_dispatch = True
                    self.empty_tray_wait = 0  # Reset the counter
                    print(f"Clock Tick {tick}: Robot {self.id} finished waiting, returning to dispatch with out-tray: {self.out_tray}")
        return

    def assign_order(self, order_id, station_id, tick):
        self.in_tray.put((order_id, station_id))
        self.proccessing_time[order_id] = tick


def generate_orders(tick):
    global order_id, stations, order_queue # Declaring global variable
    num_orders = random.randint(8, 10) #Variable count of orders generated per clock tick
    available_stations = [s for s in stations if s > 1]  # Exclude stations 0 and 1
    for i in range(num_orders):
        station_id = random.choice(available_stations) #Selects a random station from the list of available stations
        print(f"Clock Tick {tick}: New Order {order_id} generated at Station {station_id}")
        order_queue.append((order_id, station_id)) # Adds the order ID and the station ID to the order queue
        order_id += 1 #Increments order ID for the next order


def assign_orders_to_robots(robots, warehouse_graph, tick):
    global order_queue #Declaring global variable
    if not order_queue:
        return

    available_robots = [robot for robot in robots if robot.in_tray.qsize() < MAX_OUT_TRAY_CAPACITY] #Lists all the robots with set conditions to be available

    if available_robots:
        order_id, station_id = order_queue.pop(0) # Get first order in the order queue
        nearest_robot = min(
            available_robots,
            key=lambda r: nx.astar_path_length(warehouse_graph, r.current_position, station_id, weight='weight') #Finds the nearest available robots from the station considering the weight of the edge as well
        )
        nearest_robot.assign_order(order_id, station_id, tick)
        print(f"Clock Tick {tick}: Assigned Order {order_id} to Robot {nearest_robot.id} In-Tray = {list(nearest_robot.in_tray.queue)} Out-Tray = {nearest_robot.out_tray}")
    else:
        print(f"Clock Tick {tick}: No available robots to handle orders. Current order queue size: {len(order_queue)}")


def simulate():
    global tick
    total_orders = 0
    total_time = 0
    G = create_warehouse_graph()
    robots = [Robot(i, 1) for i in range(1, NUM_ROBOTS + 1)]

    for tick in range(1, SIMULATION_TIME + 1):
        generate_orders(tick)  # Generate one order per tick
        assign_orders_to_robots(robots, G, tick) # Assign orders per tick

        for robot in robots:
            robot.process_orders(G, tick) # Move robots one step

        print("\n")
        # Stop simulation early if all robots are idle and there are no orders in queue
        if all(robot.in_tray.empty() and not robot.out_tray for robot in robots) and not order_queue:
            print(f"Clock Tick {tick}: No pending orders, ending simulation.")
            break

    print("\nSimulation complete.")
    for id, time in robot_timings.items():
        if time:
            avg = sum(time)/ len(time)
            total_orders += len(time)
            total_time += avg
            print(f"Robot {id}")
            print(f"  Average Processing Time: {avg:.2f} ticks")

    print(f"Total orders completed in 24-hour cycle: {total_orders} and the average processing time per robot would be {total_time/NUM_ROBOTS:.2f} ticks")

# Run the simulation
simulate()

In [ ]:
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic
!apt-get install -y pandoc
!jupyter nbconvert --to pdf /content/PartB.ipynb